In [1]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Data prep

In [2]:
train = pd.read_csv('../data/train.csv')

In [3]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
def tokenize(s):
    pattern = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
    return pattern.sub(r' \1 ', s).split()

In [5]:
X = train.comment_text.values
idx = np.arange(len(X))

In [6]:
train['comment_text'].fillna("unknown", inplace=True)

# Model optimization

In [9]:
knn_pipe = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
                        min_df=3, max_df=0.9, strip_accents='unicode',
                        use_idf=1, smooth_idf=1, sublinear_tf=1, stop_words='english')),
    ('knn', KNeighborsClassifier(algorithm='kd_tree'))
    
])

parameters = {
    'knn__n_neighbors': [1]
}

In [10]:
best_models = {}

for i, col in enumerate(train.columns[2:]):
    print("Training ", col)
    y = train.loc[:, col].values
    clf = GridSearchCV(knn_pipe, parameters, )
    clf.fit(X, y)
    print(clf.best_model_)
    print(clf.cv_results_)
    best_models[col] = clf.best_model_
    break

Training  toxic


/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/neighbors/base.py:212: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


MemoryError: 

In [ ]:
print('Done!')